# Day 17
## Part 1

In [5]:
import parse

def parse_data(s):
    return parse.parse('target area: x={min_x:d}..{max_x:d}, y={min_y:d}..{max_y:d}', s).named

test_data = parse_data('target area: x=20..30, y=-10..-5')
test_data

{'min_x': 20, 'max_x': 30, 'min_y': -10, 'max_y': -5}

Assume that as $x$ will get to $0$ and stay there, $y$ will continue rising and fall while $x$ is constant.

$x$ and $y$ are independent, so find the initial velocities of $x$ that reach 0 between `min_x` and `max_x` and how many steps it takes them to do so. An initial velocity of $v_x$ will reach 0 after $v_x$ steps and will have an $x$ coordinate of $v_x(v_x+1)/2$, so the bounds for the velocities are given by $n^2 + n - 2x_{min}=0$ and $n^2 + n - 2x_{max}=0$. Solving the polynomials gives the following.

In [21]:
import math

def possible_x_velocities(min_x, max_x):
    return math.ceil((-1 + math.sqrt(1 + 8 * min_x)) / 2), math.floor((-1 + math.sqrt(1 + 8 * max_x)) / 2)
    

In [30]:
possible_x_velocities(20, 30)

(6, 7)

I've just realised we don't really care what $v_x$ is. Never mind.

A starting $y$ velocity of $v_y$ means $y$ reach its apex $v_y(v_y+1)/2$ after $v_y$ steps, and then takes another $v_y$ steps to return to 0, where it has a velocity of $-v_y$. 

Oh hang on, we then need to work out the highest value of $v_y$ such that its next step $-v_y-1$ is within the target area, which is 9 for the example above, and $9(9+1)/2 = 45$.

My data is `target area: x=111..161, y=-154..-101`, so

In [27]:
153*154//2

11781

is the answer.

## Part 2

Now need to find the number of pairs of velocities that land in the target at any point. For each number of steps count the number of initial $v_x$s and the number of initial $v_y$s that are in the target at that point, and multiply them together as they're independent. 

Possible values of $v_x$ are bound by the minimum value returned by the `possible_x_velocities` function - it's actually useful! - and $x_{max}$. 

$v_y$ is bound between $y_{min}$ and $-y_{min}-1$.

The $x$ values bounded by the `possible_x_velocities` function will always be there, so adjustments need to be made for that.

In [60]:
from collections import defaultdict
from itertools import product

def part_2(data):
    min_x, max_x, min_y, max_y = data['min_x'], data['max_x'], data['min_y'], data['max_y']
    n_steps_xs = defaultdict(set)
    static_xs = possible_x_velocities(min_x, max_x)
    for vx in range(static_xs[0], max_x + 1):
        x = 0
        v = vx
        n = 0
        while x < max_x and v > 0:
            x += v
            v -= 1
            n += 1
            if min_x <= x <= max_x:
                n_steps_xs[n].add(vx)
    
    n_steps_ys = defaultdict(set)
    for vy in range(min_y, -min_y):
        y = 0
        v = vy
        n = 0
        while y > min_y:
            y += v
            v -= 1
            n += 1
            if min_y <= y <= max_y:
                n_steps_ys[n].add(vy)
                
    for vx in static_xs:
        for n in n_steps_ys:
            if n > vx:
                n_steps_xs[n].add(vx)

    velocities = set()
    for n in n_steps_ys:
        for vx, vy in product(n_steps_xs[n], n_steps_ys[n]):
            velocities.add((vx, vy))

    return len(velocities)

assert part_2(test_data) == 112

In [61]:
data = parse_data('target area: x=111..161, y=-154..-101')
part_2(data)

4403

Another one where the test data passes but the real data doesn't.

Try brute-forcing both at once.

In [73]:
def part_2(data):
    min_x, max_x, min_y, max_y = data['min_x'], data['max_x'], data['min_y'], data['max_y']
    n_velocities = 0
    static_xs = possible_x_velocities(min_x, max_x)
    for vx in range(static_xs[0], max_x + 1):
        for vy in range(min_y, -min_y):
            x = 0
            y = 0
            v_x = vx
            v_y = vy
            while x <= max_x and y >= min_y:
                if x >= min_x and y <= max_y:
                    n_velocities += 1
                    break
                x += v_x
                if v_x > 0:
                    v_x -= 1
                y += v_y
                v_y -= 1

    return n_velocities

assert part_2(test_data) == 112

In [74]:
part_2(data)

4531

That's correct and ran quickly, I was trying to be too clever and made a mistake somewhere. (Solved: see post-mortem.)

In [83]:
%%timeit
part_2(data)

28.1 ms ± 202 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Post-mortem debugging

In [84]:
def part_2_a(data):
    min_x, max_x, min_y, max_y = data['min_x'], data['max_x'], data['min_y'], data['max_y']
    n_steps_xs = defaultdict(set)
    static_xs = possible_x_velocities(min_x, max_x)
    for vx in range(static_xs[0], max_x + 1):
        x = 0
        v = vx
        n = 0
        while x < max_x and v > 0:
            x += v
            v -= 1
            n += 1
            if min_x <= x <= max_x:
                n_steps_xs[n].add(vx)
    
    n_steps_ys = defaultdict(set)
    for vy in range(min_y, -min_y):
        y = 0
        v = vy
        n = 0
        while y > min_y:
            y += v
            v -= 1
            n += 1
            if min_y <= y <= max_y:
                n_steps_ys[n].add(vy)
                
    for vx in static_xs:
        for n in n_steps_ys:
            if n > vx:
                n_steps_xs[n].add(vx)

    velocities = set()
    for n in n_steps_ys:
        for vx, vy in product(n_steps_xs[n], n_steps_ys[n]):
            velocities.add((vx, vy))

    return velocities

def part_2_b(data):
    min_x, max_x, min_y, max_y = data['min_x'], data['max_x'], data['min_y'], data['max_y']
    velocities = set()
    static_xs = possible_x_velocities(min_x, max_x)
    for vx in range(static_xs[0], max_x + 1):
        for vy in range(min_y, -min_y):
            x = 0
            y = 0
            v_x = vx
            v_y = vy
            while x <= max_x and y >= min_y:
                if x >= min_x and y <= max_y:
                    velocities.add((vx, vy))
                    break
                x += v_x
                if v_x > 0:
                    v_x -= 1
                y += v_y
                v_y -= 1

    return velocities

part_2_b(data) - part_2_a(data)

{(16, 2),
 (16, 3),
 (16, 4),
 (16, 5),
 (16, 6),
 (16, 7),
 (16, 8),
 (16, 9),
 (16, 10),
 (16, 11),
 (16, 12),
 (16, 13),
 (16, 14),
 (16, 15),
 (16, 16),
 (16, 17),
 (16, 18),
 (16, 19),
 (16, 20),
 (16, 21),
 (16, 22),
 (16, 23),
 (16, 24),
 (16, 25),
 (16, 26),
 (16, 27),
 (16, 28),
 (16, 29),
 (16, 30),
 (16, 31),
 (16, 32),
 (16, 33),
 (16, 34),
 (16, 35),
 (16, 36),
 (16, 37),
 (16, 38),
 (16, 39),
 (16, 40),
 (16, 41),
 (16, 42),
 (16, 43),
 (16, 44),
 (16, 45),
 (16, 46),
 (16, 47),
 (16, 48),
 (16, 49),
 (16, 50),
 (16, 51),
 (16, 52),
 (16, 53),
 (16, 54),
 (16, 55),
 (16, 56),
 (16, 57),
 (16, 58),
 (16, 59),
 (16, 60),
 (16, 61),
 (16, 62),
 (16, 63),
 (16, 64),
 (16, 65),
 (16, 66),
 (16, 67),
 (16, 68),
 (16, 69),
 (16, 70),
 (16, 71),
 (16, 72),
 (16, 73),
 (16, 74),
 (16, 75),
 (16, 100),
 (16, 101),
 (16, 102),
 (16, 103),
 (16, 104),
 (16, 105),
 (16, 106),
 (16, 107),
 (16, 108),
 (16, 109),
 (16, 110),
 (16, 111),
 (16, 112),
 (16, 113),
 (16, 114),
 (16, 115),
 (

In [82]:
possible_x_velocities(111, 161)

(15, 17)

Ah, I should be using a range.

In [85]:
def part_2(data):
    min_x, max_x, min_y, max_y = data['min_x'], data['max_x'], data['min_y'], data['max_y']
    n_steps_xs = defaultdict(set)
    static_xs = possible_x_velocities(min_x, max_x)
    for vx in range(static_xs[0], max_x + 1):
        x = 0
        v = vx
        n = 0
        while x < max_x and v > 0:
            x += v
            v -= 1
            n += 1
            if min_x <= x <= max_x:
                n_steps_xs[n].add(vx)
    
    n_steps_ys = defaultdict(set)
    for vy in range(min_y, -min_y):
        y = 0
        v = vy
        n = 0
        while y > min_y:
            y += v
            v -= 1
            n += 1
            if min_y <= y <= max_y:
                n_steps_ys[n].add(vy)
                
    for vx in range(static_xs[0], static_xs[1] + 1):
        for n in n_steps_ys:
            if n > vx:
                n_steps_xs[n].add(vx)

    velocities = set()
    for n in n_steps_ys:
        for vx, vy in product(n_steps_xs[n], n_steps_ys[n]):
            velocities.add((vx, vy))

    return len(velocities)

assert part_2(test_data) == 112

In [86]:
part_2(data)

4531

In [87]:
%%timeit
part_2(data)

3.1 ms ± 15.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Quite a bit faster.